In [1]:
#Preprocessing
# Import pustaka yang diperlukan
import pandas as pd

# Path file CSV
file_path = "data_kasus_dbd_dki_jakarta_2015_2020.csv"  # Ganti dengan nama file CSV Anda

# Membaca file CSV
try:
    data = pd.read_csv(file_path)
    print("File berhasil dibaca.")

    # Menampilkan lima baris pertama dari data awal
    print("Data awal:")
    print(data.head())

    # Memastikan kolom 'periode' dan 'jumlah_kasus' ada
    if 'periode' in data.columns and 'jumlah_kasus' in data.columns:
        # Memisahkan kolom 'periode' dan 'jumlah_kasus'
        df_preprocessed = data[['periode', 'jumlah_kasus']].copy()
        
        # Menampilkan hasil preprocessing
        print("\nData setelah preprocessing:")
        print(df_preprocessed.head())
    else:
        print("Kolom 'periode' atau 'jumlah_kasus' tidak ditemukan dalam data.")
except FileNotFoundError:
    print(f"File {file_path} tidak ditemukan. Pastikan file ada di direktori yang sesuai.")


File data_kasus_dbd_dki_jakarta_2015_2020.csv tidak ditemukan. Pastikan file ada di direktori yang sesuai.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# Load Data
data = pd.read_csv( r"C:\\Users\\ASUS\\Downloads\\informer\\augmented_dbd_cases.csv")  # Ganti dengan path dataset Anda
data["periode"] = pd.to_datetime(data["periode"], format="%Y-%m")
data.set_index("periode", inplace=True)

# Normalisasi Data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[["jumlah_kasus"]])

# Fungsi untuk Membuat Windowed Dataset
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Hyperparameter
SEQ_LEN = 12  # Menggunakan data 12 bulan sebelumnya untuk prediksi
BATCH_SIZE = 16
EPOCHS = 50
DROPOUT_RATE = 0.2

# Siapkan Dataset
X, y = create_sequences(data_scaled, SEQ_LEN)

# Split Data untuk Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

# Simpan Evaluasi untuk Setiap Fold
evaluation_results = []

for train_index, test_index in kf.split(X):
    print(f"Fold {fold}:")
    
    # Train-Test Split
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Model Arsitektur
    model = Sequential([
        LSTM(64, activation="relu", return_sequences=True, input_shape=(SEQ_LEN, 1)),
        Dropout(DROPOUT_RATE),
        LSTM(32, activation="relu"),
        Dropout(DROPOUT_RATE),
        Dense(1)
    ])
    
    model.compile(optimizer="adam", loss="mean_squared_error")
    
    # Pelatihan Model
    history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, validation_data=(X_test, y_test))
    
    # Prediksi
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    actuals = scaler.inverse_transform(y_test.reshape(-1, 1))
    
    # Evaluasi
    mse = mean_squared_error(actuals, predictions)
    rmse = mse ** 0.5
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)
    mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100  # Evaluasi MAPE
    
    evaluation_results.append((mse, rmse, mae, r2, mape))
    print(f"MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}, MAPE: {mape:.4f}%")
    fold += 1

# Visualisasi Hasil Fold Terakhir
plt.figure(figsize=(12, 6))
plt.plot(actuals, label="Actual")
plt.plot(predictions, label="Predicted")
plt.title("Prediksi vs Data Aktual (Fold Terakhir)")
plt.xlabel("Waktu")
plt.ylabel("Jumlah Kasus")
plt.legend()
plt.show()

# Rata-rata Evaluasi Cross-Validation
results_df = pd.DataFrame(evaluation_results, columns=["MSE", "RMSE", "MAE", "R2", "MAPE"])
print("Hasil Rata-Rata Cross-Validation:")
print(results_df.mean())


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# Load Data
data = pd.read_csv(r"C:\\Users\\ASUS\\Downloads\\informer\\augmented_dbd_cases.csv")  # Ganti dengan path dataset Anda
data["periode"] = pd.to_datetime(data["periode"], format="%Y-%m")
data.set_index("periode", inplace=True)

# Normalisasi Data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[["jumlah_kasus"]])

# Fungsi untuk Membuat Windowed Dataset
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

# Hyperparameter
SEQ_LEN = 12  # Menggunakan data 12 bulan sebelumnya untuk prediksi
BATCH_SIZE = 16
EPOCHS = 50
DROPOUT_RATE = 0.2

# Siapkan Dataset
X, y = create_sequences(data_scaled, SEQ_LEN)

# Split Data untuk Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

# Buat folder untuk menyimpan model jika belum ada
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

# Simpan Evaluasi untuk Setiap Fold
evaluation_results = []

for train_index, test_index in kf.split(X):
    print(f"Fold {fold}:")
    
    # Train-Test Split
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Model Arsitektur
    model = Sequential([
        LSTM(64, activation="relu", return_sequences=True, input_shape=(SEQ_LEN, 1)),
        Dropout(DROPOUT_RATE),
        LSTM(32, activation="relu"),
        Dropout(DROPOUT_RATE),
        Dense(1)
    ])
    
    model.compile(optimizer="adam", loss="mean_squared_error")
    
    # Pelatihan Model
    history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1, validation_data=(X_test, y_test))
    
    # Simpan Model Setelah Pelatihan
    model_save_path = os.path.join(save_dir, f"model_fold_{fold}.h5")
    model.save(model_save_path)
    print(f"Model Fold {fold} disimpan di {model_save_path}")
    
    # Prediksi
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    actuals = scaler.inverse_transform(y_test.reshape(-1, 1))
    
    # Evaluasi
    mse = mean_squared_error(actuals, predictions)
    rmse = mse ** 0.5
    mae = mean_absolute_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)
    mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100  # Evaluasi MAPE
    
    evaluation_results.append((mse, rmse, mae, r2, mape))
    print(f"MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}, MAPE: {mape:.4f}%")
    fold += 1

# Visualisasi Hasil Fold Terakhir
plt.figure(figsize=(12, 6))
plt.plot(actuals, label="Actual")
plt.plot(predictions, label="Predicted")
plt.title("Prediksi vs Data Aktual (Fold Terakhir)")
plt.xlabel("Waktu")
plt.ylabel("Jumlah Kasus")
plt.legend()
plt.show()

# Rata-rata Evaluasi Cross-Validation
results_df = pd.DataFrame(evaluation_results, columns=["MSE", "RMSE", "MAE", "R2", "MAPE"])
print("Hasil Rata-Rata Cross-Validation:")
print(results_df.mean())
